In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


# 1. Data Pre-Processing

In [2]:
shortlist = list(pd.read_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/shortlist_for_ML_Opti_5ppm.csv")['features'])

In [3]:
df_ROI_for_ML_Opti_train = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_5ppm/df5ppm_ROI4_for_ML_Opti_train.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_ext = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_5ppm/df5ppm_ROI4_for_ML_Opti_ext.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_ingested = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_5ppm/df5ppm_ROI4_for_ML_Opti_ingested.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_FNA = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_5ppm/df5ppm_ROI4_for_ML_Opti_FNA.csv").set_index('pixel_id')[shortlist]
df_ROI_for_ML_Opti_DirectIn = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_DirectIn/0_dbMSI_5ppm/df5ppm_ROI4_for_ML_Opti_DI.csv").set_index('pixel_id')[shortlist]

In [4]:
df_pixels_train = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_raMSIn4nonInDI.csv")
pixels_train = list(df_pixels_train["pixel_id"])
df_pixels_ext = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_raMSIn4nonInDI.csv")
pixels_ext = list(df_pixels_ext["pixel_id"])
df_pixels_ingested = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_raMSIn4nonInDI.csv")
pixels_ingested = list(df_pixels_ingested["pixel_id"])
df_pixels_FNA = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_raMSIn4nonInDI.csv")
pixels_FNA = list(df_pixels_FNA["pixel_id"])

In [5]:
df_ROI_for_ML_Opti_train=df_ROI_for_ML_Opti_train.reset_index()
df_ROI_for_ML_Opti_ext=df_ROI_for_ML_Opti_ext.reset_index()
df_ROI_for_ML_Opti_ingested=df_ROI_for_ML_Opti_ingested.reset_index()
df_ROI_for_ML_Opti_FNA=df_ROI_for_ML_Opti_FNA.reset_index()
df_ROI_for_ML_Opti_DirectIn=df_ROI_for_ML_Opti_DirectIn.reset_index()

In [6]:
df_ROI_for_ML_Opti_train = df_ROI_for_ML_Opti_train.loc[df_ROI_for_ML_Opti_train["pixel_id"].isin(pixels_train)].set_index("pixel_id")
df_ROI_for_ML_Opti_ext = df_ROI_for_ML_Opti_ext.loc[df_ROI_for_ML_Opti_ext["pixel_id"].isin(pixels_ext)].set_index("pixel_id")
df_ROI_for_ML_Opti_ingested = df_ROI_for_ML_Opti_ingested.loc[df_ROI_for_ML_Opti_ingested["pixel_id"].isin(pixels_ingested)].set_index("pixel_id")
df_ROI_for_ML_Opti_FNA = df_ROI_for_ML_Opti_FNA.loc[df_ROI_for_ML_Opti_FNA["pixel_id"].isin(pixels_FNA)].set_index("pixel_id")
df_ROI_for_ML_Opti_DirectIn = df_ROI_for_ML_Opti_DirectIn.set_index("pixel_id")

In [7]:
df_ROI_for_ML_Opti_train

,124.007,179.056,195.0508,215.0328,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2239.475098,5285.062012,1357.171875,1765.094360,1203.201172,3446.257080,1359.884644,10417.782227,9099.804688,25930.855469,...,0.000000,4257.063965,0.0,0.000000,0.0,2001.672241,31168.623047,15905.403320,9028.520508,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4155.390625,6133.164062,1921.501221,2892.591553,1378.336182,5497.271484,1761.496948,19751.246094,10752.050781,27755.326172,...,0.000000,4487.459473,0.0,0.000000,0.0,3280.133423,30545.011719,14919.373047,9713.387695,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1292.693970,3695.261230,0.000000,2225.103027,1561.307861,6164.573242,1269.849487,14473.411133,14948.118164,34604.625000,...,0.000000,3653.516846,0.0,0.000000,0.0,2032.922119,33217.960938,15143.654297,9782.683594,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1242.931152,3635.843750,488.741669,2827.721436,1062.043701,5087.580078,1637.160278,13347.621094,11885.568359,31051.693359,...,584.368896,2427.404541,0.0,0.000000,0.0,2229.916992,35512.621094,16859.195312,9062.646484,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2048.168701,5988.678223,809.635071,1611.982788,1233.044678,2927.715088,1039.213745,9052.196289,9366.003906,25871.083984,...,0.000000,3240.175293,0.0,608.407104,0.0,2252.401855,33546.597656,17407.480469,8189.253418,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_106_77,3674.323730,16039.344727,2610.340332,3595.426758,959.914185,3322.557617,0.000000,6830.675293,7581.434570,25921.128906,...,0.000000,1591.155884,0.0,0.000000,0.0,1217.228271,14541.902344,7881.114258,4629.981445,1
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_40_78,2028.606812,8517.287109,0.000000,2122.539551,964.505127,2835.578125,970.017761,14570.761719,9153.279297,24488.681641,...,0.000000,1300.384399,0.0,1186.961548,0.0,2125.813721,16925.468750,10088.454102,5096.657715,1
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW_41_78,2581.488281,10185.144531,0.000000,2093.726318,1103.738281,4479.292969,0.000000,11204.238281,7049.967285,31199.921875,...,0.000000,2044.695801,0.0,1405.690674,0.0,1770.959473,23411.234375,12572.214844,7795.006836,1


In [8]:
df_ROI_for_ML_Opti_DirectIn

,124.007,179.056,195.0508,215.0328,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_1_1,5.547676e+06,263134.21880,7.052622e+05,309304.9688,6.823191e+05,7.206863e+05,5.719936e+05,1.326359e+06,2.340093e+06,23934682.00,...,163518.21880,224169.04690,0.00000,270663.3750,0.00000,150088.20310,7.726446e+05,318562.4688,166946.42190,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_2_1,8.226763e+06,427196.15630,8.734356e+05,597479.6875,6.796848e+05,8.966212e+05,4.892583e+05,1.936425e+06,2.489642e+06,29897656.00,...,308611.90630,352169.21880,66618.60156,665543.9375,167263.57810,235876.03130,1.331586e+06,637292.5625,216978.07810,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_3_1,8.064516e+06,419983.53130,9.750671e+05,300903.5938,6.645241e+05,6.950204e+05,6.188099e+05,1.316102e+06,2.146926e+06,24283230.00,...,101337.92970,181263.60940,0.00000,330792.5000,45643.00391,190057.67190,7.520046e+05,376333.9375,181019.17190,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_4_1,1.021061e+07,486953.03130,1.142831e+06,351061.8438,9.616746e+05,9.388458e+05,7.611674e+05,1.753238e+06,3.078821e+06,35782728.00,...,136772.09380,396866.03130,0.00000,459988.9063,78579.17969,196964.87500,1.451490e+06,592289.4375,350863.87500,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_5_1,1.172199e+07,598518.68750,1.298470e+06,543811.6875,1.237240e+06,1.225918e+06,1.089950e+06,2.311059e+06,3.571950e+06,43148492.00,...,378720.15630,356264.90630,63569.64844,874194.7500,257116.73440,319688.31250,1.853836e+06,940595.8125,273320.56250,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_602_1,1.303348e+07,345843.71880,2.414753e+06,486091.4688,4.207686e+05,1.373586e+05,1.480586e+05,0.000000e+00,1.659227e+06,17565150.00,...,199226.46880,183160.45310,0.00000,462036.6250,140198.09380,64716.57813,8.224419e+05,351214.2188,136051.06250,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_603_1,1.027775e+06,95467.11719,4.669226e+05,0.0000,8.158860e+04,1.201855e+05,5.609006e+04,1.760277e+04,2.385627e+05,2729922.25,...,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.000000e+00,0.0000,0.00000,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_604_1,9.098884e+06,300681.06250,2.176268e+06,242408.1875,3.346165e+05,5.971110e+04,1.454526e+05,0.000000e+00,1.189476e+06,13479287.00,...,139478.59380,55146.58594,0.00000,220301.8281,0.00000,63270.16406,5.020935e+05,265960.1875,116673.78130,1


In [9]:
df_ROI_train = df_ROI_for_ML_Opti_train.reset_index()
df_ROI_ext = df_ROI_for_ML_Opti_ext.reset_index()
df_ROI_ingested = df_ROI_for_ML_Opti_ingested.reset_index()
df_ROI_FNA = df_ROI_for_ML_Opti_FNA.reset_index()
df_ROI_DirectIn = df_ROI_for_ML_Opti_DirectIn.reset_index()

In [10]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_ingested["Sample"] = 0
df_ROI_ingested["Row"] = 0
df_ROI_ingested["Scan"] = 0
df_ROI_ingested_Sample = []
df_ROI_ingested_Row = []
df_ROI_ingested_Scan = []
for i in range(len(df_ROI_ingested["pixel_id"])):
    df_ROI_ingested_Sample.append("_".join(str(df_ROI_ingested["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ingested_Row.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-1]))
    df_ROI_ingested_Scan.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-2]))
df_ROI_ingested["Sample"] = df_ROI_ingested_Sample
df_ROI_ingested["Row"] = df_ROI_ingested_Row
df_ROI_ingested["Scan"] = df_ROI_ingested_Scan
pop_column = df_ROI_ingested.pop('Scan')
df_ROI_ingested.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ingested.pop('Row')
df_ROI_ingested.insert(1, 'Row', pop_column)
pop_column = df_ROI_ingested.pop('Sample')
df_ROI_ingested.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_DirectIn["Sample"] = 0
df_ROI_DirectIn["Row"] = 0
df_ROI_DirectIn["Scan"] = 0
df_ROI_DirectIn_Sample = []
df_ROI_DirectIn_Row = []
df_ROI_DirectIn_Scan = []
for i in range(len(df_ROI_DirectIn["pixel_id"])):
    df_ROI_DirectIn_Sample.append("_".join(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[:-2]))
    df_ROI_DirectIn_Row.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-1]))
    df_ROI_DirectIn_Scan.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-2]))
df_ROI_DirectIn["Sample"] = df_ROI_DirectIn_Sample
df_ROI_DirectIn["Row"] = df_ROI_DirectIn_Row
df_ROI_DirectIn["Scan"] = df_ROI_DirectIn_Scan
pop_column = df_ROI_DirectIn.pop('Scan')
df_ROI_DirectIn.insert(1, 'Scan', pop_column)
pop_column = df_ROI_DirectIn.pop('Row')
df_ROI_DirectIn.insert(1, 'Row', pop_column)
pop_column = df_ROI_DirectIn.pop('Sample')
df_ROI_DirectIn.insert(1, 'Sample', pop_column)

In [11]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ingested = df_ROI_ingested.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_DirectIn = df_ROI_DirectIn.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [12]:
df_ROI_DirectIn

,pixel_id,Sample,Row,Scan,124.007,179.056,195.0508,215.0328,227.2016,233.1546,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,1,5.547676e+06,263134.21880,7.052622e+05,309304.96880,6.823191e+05,720686.31250,...,163518.21880,224169.0469,0.00000,270663.3750,0.00000,150088.2031,7.726446e+05,318562.4688,166946.42190,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,8.388430e+06,511173.09380,8.645568e+05,488069.84380,1.168717e+06,804839.81250,...,73133.15625,238254.5625,77866.75781,564040.0625,188116.31250,184324.8125,1.380952e+06,592850.5625,223408.31250,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,401,7.579751e+06,440915.43750,8.726503e+05,514671.56250,8.598869e+05,892335.37500,...,252651.81250,161097.4844,0.00000,278442.3125,83146.02344,223186.6250,1.135078e+06,423860.4375,197581.54690,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,402,8.341370e+06,438205.56250,1.002705e+06,391519.75000,8.872040e+05,772144.18750,...,226212.93750,248052.3438,139624.34380,563859.2500,142363.84380,227635.4063,9.275429e+05,484964.6875,213537.39060,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,403,8.079836e+06,507348.71880,9.420554e+05,317669.71880,1.077575e+06,908982.75000,...,285310.15630,335359.9063,0.00000,614402.0000,72634.45313,197951.3125,1.054263e+06,486644.4688,211347.70310,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,203,4.396891e+05,27796.03516,1.660753e+05,0.00000,9.744127e+04,133731.40630,...,0.00000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000e+00,0.0000,0.00000,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,204,6.532192e+06,250603.90630,1.200827e+06,46412.19531,2.601084e+05,150229.18750,...,14078.69727,0.0000,0.00000,0.0000,0.00000,0.0000,1.634594e+04,0.0000,0.00000,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,205,1.959175e+07,967865.75000,3.347390e+06,595004.56250,6.028912e+05,145226.40630,...,65635.39063,115989.1719,34096.28906,259049.7813,66982.69531,85668.6250,3.841148e+05,163241.2188,88670.65625,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,207,8.674445e+05,91974.18750,3.387656e+05,0.00000,1.226611e+05,135125.14060,...,0.00000,0.0000,0.00000,0.0000,0.00000,0.0000,0.000000e+00,0.0000,0.00000,1


In [13]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ingested.loc[df_ROI_ingested["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_DirectIn.loc[df_ROI_DirectIn["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [14]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [15]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

In [16]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [17]:
def ROI_Extract(sample_):
    TOP ="885.5488"
    preview = sample_[[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)
    vmin_list = list(preview.describe().iloc[-3])
    vmin_ = vmin_list[0]
    sample_ = sample_.loc[sample_["885.5488"] > vmin_].reset_index().drop(columns=["index"])

    return sample_

In [18]:
sample_ROI_list_DirectIn_list = []

for sample in sample_ROI_list_DirectIn:
    
    df_ROI = ROI_Extract(sample_=sample)
    sample_ROI_list_DirectIn_list.append(df_ROI)

C:\Users\user\AppData\Local\Temp\ipykernel_23516\3560934736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preview[i] = preview[i].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_23516\3560934736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preview[i] = preview[i].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_23516\3560934736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [19]:
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = sample_ROI_list_DirectIn_list[0]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = sample_ROI_list_DirectIn_list[1]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = sample_ROI_list_DirectIn_list[2]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = sample_ROI_list_DirectIn_list[3]
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = sample_ROI_list_DirectIn_list[4]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = sample_ROI_list_DirectIn_list[5]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = sample_ROI_list_DirectIn_list[6]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = sample_ROI_list_DirectIn_list[7]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = sample_ROI_list_DirectIn_list[8]
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = sample_ROI_list_DirectIn_list[9]

In [20]:
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI

,pixel_id,Sample,Row,Scan,124.007,179.056,195.0508,215.0328,227.2016,233.1546,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,2,8226763.0,427196.1563,8.734356e+05,597479.6875,6.796848e+05,8.966212e+05,...,308611.9063,352169.2188,66618.60156,6.655439e+05,167263.57810,235876.0313,1331585.625,6.372926e+05,216978.0781,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,4,10210607.0,486953.0313,1.142831e+06,351061.8438,9.616746e+05,9.388458e+05,...,136772.0938,396866.0313,0.00000,4.599889e+05,78579.17969,196964.8750,1451490.000,5.922894e+05,350863.8750,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,5,11721988.0,598518.6875,1.298470e+06,543811.6875,1.237240e+06,1.225918e+06,...,378720.1563,356264.9063,63569.64844,8.741948e+05,257116.73440,319688.3125,1853836.250,9.405958e+05,273320.5625,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,9,11873774.0,524780.1875,1.397608e+06,423693.0313,1.173549e+06,8.920201e+05,...,163378.0625,306777.6875,138555.59380,5.200097e+05,180495.23440,345452.5625,1594346.000,7.149868e+05,310046.5938,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,11,10862974.0,782438.6250,1.406976e+06,391590.7188,1.114541e+06,1.054546e+06,...,164940.6406,254745.5156,0.00000,5.643072e+05,192787.12500,246124.2188,1256030.875,6.319585e+05,204829.1563,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,595,9092751.0,738350.8125,1.107328e+06,704786.3750,1.590076e+06,8.988187e+05,...,200590.0000,348169.3438,215933.50000,1.377972e+06,344897.65630,225340.1875,1579836.875,1.097872e+06,339542.7500,0
297,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,596,10097616.0,586149.5625,1.002128e+06,332230.1875,1.343452e+06,8.545741e+05,...,273935.1250,349874.2500,0.00000,6.244413e+05,235178.15630,270488.1250,1550177.125,6.325014e+05,268800.0938,0
298,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,598,10767637.0,593391.2500,1.173724e+06,500021.6563,1.246096e+06,9.850924e+05,...,240068.0781,254313.5625,0.00000,8.760864e+05,270689.93750,271963.9063,1346329.875,7.646786e+05,237992.1563,0
299,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,600,10176433.0,458297.6250,9.392391e+05,327043.7500,1.076830e+06,8.701148e+05,...,358613.2500,491605.6563,313109.68750,8.865479e+05,186632.43750,273641.5000,1440054.750,9.782394e+05,301364.7188,0


In [21]:
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI

,pixel_id,Sample,Row,Scan,124.007,179.056,195.0508,215.0328,227.2016,233.1546,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
0,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,1,12176738.0,209030.1719,1728867.000,258694.98440,441864.5000,189055.10940,...,308745.90630,384737.87500,50452.91406,2.564622e+06,735014.5000,188071.89060,1.392550e+06,753209.3750,322619.65630,1
1,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,2,14135779.0,269538.7500,2509547.750,251050.28130,524995.7500,195003.90630,...,305575.00000,205010.51560,0.00000,1.715896e+06,592634.5000,197198.89060,1.173196e+06,534890.7500,58295.61328,1
2,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,11,11700827.0,190732.6719,2045673.625,258700.06250,361530.7188,161930.18750,...,200480.70310,203869.42190,0.00000,1.845380e+06,511332.2500,92970.92188,8.755424e+05,527000.5000,136271.57810,1
3,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,14,11333077.0,190603.4219,1513526.375,184270.09380,314123.2500,52518.69141,...,193799.09380,157180.79690,0.00000,1.265864e+06,351378.7188,53579.78906,7.414910e+05,286182.3438,48463.08984,1
4,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,15,13524384.0,269026.5313,1983786.875,183153.67190,438186.0625,165417.37500,...,258517.64060,53684.16797,0.00000,7.157385e+05,196756.5781,63260.31250,7.550344e+05,405735.1875,118373.39060,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,583,11499414.0,176470.2500,1430460.250,46794.98438,324425.0625,152517.48440,...,194974.93750,75734.00781,0.00000,8.987371e+05,190348.0938,0.00000,7.573950e+05,162382.7813,0.00000,1
299,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,589,8563978.0,182286.5938,1462631.000,49274.76172,258134.0313,175414.01560,...,244849.73440,61469.36328,0.00000,1.339231e+06,560152.6250,131856.20310,8.186472e+05,383421.5313,230939.51560,1
300,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,602,10670922.0,248095.2813,1559295.375,62621.28125,341265.5938,185168.98440,...,327466.84380,134935.50000,0.00000,1.808776e+06,536243.5000,151135.35940,9.289883e+05,502995.8438,181200.14060,1
301,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,604,12146854.0,149882.1719,1951225.250,132508.60940,335704.6563,159219.56250,...,167874.57810,302721.81250,45184.06641,2.496261e+06,736308.3750,184380.67190,1.370530e+06,650568.5625,293856.18750,1


In [22]:
df_ROI_for_ML_Opti_DirectIn = pd.DataFrame()
for df in sample_ROI_list_DirectIn_list:
    df_ROI_for_ML_Opti_DirectIn = pd.concat([df_ROI_for_ML_Opti_DirectIn, df], join="outer", ignore_index=True)

In [23]:
df_ROI_for_ML_Opti_DirectIn=df_ROI_for_ML_Opti_DirectIn.drop(columns=["Sample", "Row", "Scan"]).set_index("pixel_id")

In [24]:
df_ROI_for_ML_Opti_DirectIn

,124.007,179.056,195.0508,215.0328,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_2_1,8226763.0,427196.1563,8.734356e+05,597479.6875,6.796848e+05,8.966212e+05,4.892583e+05,1.936425e+06,2.489642e+06,29897656.0,...,308611.9063,352169.2188,66618.60156,6.655439e+05,167263.57810,235876.0313,1331585.625,6.372926e+05,216978.0781,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_4_1,10210607.0,486953.0313,1.142831e+06,351061.8438,9.616746e+05,9.388458e+05,7.611674e+05,1.753238e+06,3.078821e+06,35782728.0,...,136772.0938,396866.0313,0.00000,4.599889e+05,78579.17969,196964.8750,1451490.000,5.922894e+05,350863.8750,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_5_1,11721988.0,598518.6875,1.298470e+06,543811.6875,1.237240e+06,1.225918e+06,1.089950e+06,2.311059e+06,3.571950e+06,43148492.0,...,378720.1563,356264.9063,63569.64844,8.741948e+05,257116.73440,319688.3125,1853836.250,9.405958e+05,273320.5625,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_9_1,11873774.0,524780.1875,1.397608e+06,423693.0313,1.173549e+06,8.920201e+05,8.630922e+05,1.988560e+06,3.294478e+06,38693388.0,...,163378.0625,306777.6875,138555.59380,5.200097e+05,180495.23440,345452.5625,1594346.000,7.149868e+05,310046.5938,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_11_1,10862974.0,782438.6250,1.406976e+06,391590.7188,1.114541e+06,1.054546e+06,9.458113e+05,2.105050e+06,3.820051e+06,39665900.0,...,164940.6406,254745.5156,0.00000,5.643072e+05,192787.12500,246124.2188,1256030.875,6.319585e+05,204829.1563,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_594_1,14243413.0,557954.5000,3.205326e+06,796842.0000,4.959218e+05,2.192356e+05,3.384922e+05,0.000000e+00,2.609134e+06,34368128.0,...,215761.4063,386555.9688,68249.75781,1.163364e+06,284302.46880,340221.1563,1470005.250,7.135132e+05,518926.6875,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_595_1,7453880.0,371035.1875,2.420200e+06,131669.6719,1.858623e+05,1.622703e+05,5.536682e+04,4.189609e+04,7.838960e+05,9848581.0,...,309121.2813,335784.5938,165061.51560,8.114948e+05,353163.93750,254814.9063,2130823.750,1.099195e+06,437886.0938,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_596_1,19376702.0,658093.3750,3.174590e+06,757211.8750,3.538968e+05,1.700947e+05,2.470646e+05,0.000000e+00,1.619394e+06,23178838.0,...,210538.8750,541987.1875,0.00000,5.276537e+05,193192.62500,176609.1094,1115554.000,5.783931e+05,258386.8125,1


In [25]:
df_ROI_for_ML_Opti_train.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_ROI_for_ML_Opti_train_5ppm.csv")
df_ROI_for_ML_Opti_ext.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_ROI_for_ML_Opti_ext_5ppm.csv")
df_ROI_for_ML_Opti_ingested.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_ROI_for_ML_Opti_ingested_5ppm.csv")
df_ROI_for_ML_Opti_FNA.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_ROI_for_ML_Opti_FNA_5ppm.csv")
df_ROI_for_ML_Opti_DirectIn.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_ROI_for_ML_Opti_DirectIn_5ppm.csv")

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [26]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

## 2.2. PCA Analysis

In [37]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_DirectIn):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_DirectIn.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/PCA_DirectIn_5ppm.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [38]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [39]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

In [40]:
X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti_ingested)

In [41]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [42]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [43]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [44]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [45]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[:-1]]

In [46]:
y_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[-1]]

## 3.2 Machine Learning Modeling

In [47]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.2. XBG

In [48]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings
mpl.rcParams['figure.dpi'] = 300

In [49]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal= X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2)
    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set AUROC:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_train_shisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set AUROC:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_val_shoisted.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_test = model_xgb.predict(XTest)
    print("Test Set AUROC:",metrics.roc_auc_score(yTest, predictions_test))
    print(classification_report(yTest,predictions_test))
    print(confusion_matrix(yTest,predictions_test))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_test, tpr_test, _ = metrics.roc_curve(yTest, predictions_test)
    plt.plot(fpr_test,tpr_test)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_test_shosted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp[:25], y=feature_imp[:25].index)
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_TopFeatures_stlisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [50]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.51660	validation_1-auc:0.50626
[1]	validation_0-auc:0.64241	validation_1-auc:0.47083
[2]	validation_0-auc:0.51143	validation_1-auc:0.50297
Training Set AUROC: 0.9503094374412009
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     50392
           1       0.93      0.97      0.95     46643

    accuracy                           0.95     97035
   macro avg       0.95      0.95      0.95     97035
weighted avg       0.95      0.95      0.95     97035

[[47145  3247]
 [ 1630 45013]]
Validation Set AUROC: 0.5165985949664302
              precision    recall  f1-score   support

           0       0.51      1.00      0.68     44540
           1       0.93      0.04      0.07     44161

    accuracy                           0.52     88701
   macro avg       0.72      0.52      0.37     88701
weighted avg       0.72      0.52      0.37     88701

[[44420   120]
 [42576  1585]]
Test Set AUROC: 0.5062636468537799
       

In [51]:
coeff_XGB  # coeff

311.1684    0.456382
269.2486    0.215584
215.0328    0.072590
241.2173    0.052979
295.2278    0.037948
              ...   
179.056     0.000000
328.2365    0.000000
329.2492    0.000000
351.0449    0.000000
887.5598    0.000000
Length: 84, dtype: float32

In [52]:
TOPfeatures_XGB  # features

Index(['311.1684', '269.2486', '215.0328', '241.2173', '295.2278', '435.2965',
       '480.3096', '883.533', '309.1725', '339.1996', '233.1546', '337.2055',
       '265.1478', '279.2328', '790.5408', '303.2328', '325.1842', '867.5136',
       '253.2171', '250.1449', '195.0508', '485.2796', '664.9179', '602.4455',
       '563.9381', '592.994', '666.5128', '515.2877', '514.2846', '509.3127',
       '665.7862', '124.007', '717.4791', '738.5057', '886.5521', '885.5488',
       '865.5023', '863.0912', '857.5177', '794.5338', '773.5334', '770.5053',
       '767.5368', '766.539', '763.511', '438.9672', '760.5145', '748.4925',
       '747.4937', '762.5074', '327.2328', '436.1056', '285.2071', '227.2016',
       '255.2328', '256.2363', '266.1513', '267.1965', '269.2122', '271.2278',
       '277.2172', '280.2361', '281.2484', '282.252', '283.2278', '283.2642',
       '284.2676', '293.1774', '353.2004', '297.1116', '297.153', '297.2434',
       '298.2469', '299.2592', '301.2174', '304.2364', '305

In [53]:
len(TOPfeatures_XGB)

84

In [54]:
TOP_features_XGB  # old features (dont run)

NameError: name 'TOP_features_XGB' is not defined

## 3.3. Sensitivity

In [55]:
def df_maker(features_list, df= df_ROI_for_ML_Opti_ingested):
    df_sensitivity = df.T.T.drop(columns=list(df.columns))
    for fea in features_list:
        df_sensitivity[fea] = df[fea]
    df_sensitivity["type"] = df["type"]
    return df_sensitivity

In [56]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

### 3.3.2. XGB

In [57]:
def XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    
    #model_xgb.fit(XTrain, yTrain)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    fit_params = {'eval_set': [[XVal, yVal], [XTest, yTest]]}

    scores_XGB = cross_val_score(model_xgb, XTrain, yTrain, cv=5, 
                                 scoring = 'roc_auc',
                                 fit_params = fit_params)
    
    print(scores_XGB.mean())
    print(scores_XGB.std())
    X_train_predictions = model_xgb.predict(XTrain)
    predictions_val = model_xgb.predict(XVal)
    predictions_test = model_xgb.predict(XTest)
    
    a_f1 = f1_score(yTrain, X_train_predictions, pos_label=1)
    b_f1 = f1_score(yVal, predictions_val, pos_label=1)
    c_f1 = f1_score(yTest, predictions_test, pos_label=1)
    a_auc = metrics.roc_auc_score(yTrain, X_train_predictions)
    b_auc = metrics.roc_auc_score(yVal, predictions_val)
    c_auc = metrics.roc_auc_score(yTest, predictions_test)

    cv_acr_XGB = scores_XGB.mean()
    cv_std_XGB = scores_XGB.std()

    return cv_acr_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc

In [58]:
X_ingested

,124.007,179.056,195.0508,215.0328,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,790.5408,794.5338,857.5177,863.0912,865.5023,867.5136,883.533,885.5488,886.5521,887.5598
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2239.475098,5285.062012,1357.171875,1765.094360,1203.201172,3446.257080,1359.884644,10417.782227,9099.804688,25930.855469,...,1250.714233,0.000000,4257.063965,0.0,0.000000,0.0,2001.672241,31168.623047,15905.403320,9028.520508
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4155.390625,6133.164062,1921.501221,2892.591553,1378.336182,5497.271484,1761.496948,19751.246094,10752.050781,27755.326172,...,1064.463623,0.000000,4487.459473,0.0,0.000000,0.0,3280.133423,30545.011719,14919.373047,9713.387695
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1292.693970,3695.261230,0.000000,2225.103027,1561.307861,6164.573242,1269.849487,14473.411133,14948.118164,34604.625000,...,820.255554,0.000000,3653.516846,0.0,0.000000,0.0,2032.922119,33217.960938,15143.654297,9782.683594
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1242.931152,3635.843750,488.741669,2827.721436,1062.043701,5087.580078,1637.160278,13347.621094,11885.568359,31051.693359,...,1733.640259,584.368896,2427.404541,0.0,0.000000,0.0,2229.916992,35512.621094,16859.195312,9062.646484
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2048.168701,5988.678223,809.635071,1611.982788,1233.044678,2927.715088,1039.213745,9052.196289,9366.003906,25871.083984,...,1646.036011,0.000000,3240.175293,0.0,608.407104,0.0,2252.401855,33546.597656,17407.480469,8189.253418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_37_30,424.004242,580.648621,0.000000,0.000000,2522.145996,3230.580566,1575.619995,9511.707031,663.538635,17844.457031,...,0.000000,0.000000,311.449890,0.0,0.000000,0.0,0.000000,6123.190430,2633.123535,1756.053955
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_38_30,595.734192,326.932434,240.059250,0.000000,1978.850342,3854.620850,1554.890381,9744.109375,635.373474,20401.718750,...,430.207520,0.000000,1771.128540,0.0,529.190857,0.0,1543.361328,20750.964844,10420.350586,5912.117676
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_39_30,284.465607,540.562500,0.000000,0.000000,2322.719727,3062.539062,1245.831055,9213.754883,311.672241,17421.740234,...,739.540833,0.000000,713.486450,0.0,0.000000,0.0,502.586395,12818.436035,7673.676270,3264.151123


In [59]:
len(list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns))

84

In [60]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]
f_ = [""]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

#for i in range(39):
for i in range(len(TOPfeatures_XGB)):
    print(i)
    features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
    features_list.remove(list(TOPfeatures_XGB)[i])
    f_.append(list(TOPfeatures_XGB)[i])
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    
    cv_auc_his_XGB.append(cv_auc_XGB)
    cv_std_his_XGB.append(cv_std_XGB)

    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.51660	validation_1-auc:0.50626
[1]	validation_0-auc:0.64241	validation_1-auc:0.47083
[2]	validation_0-auc:0.51143	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.50709	validation_1-auc:0.50626
[1]	validation_0-auc:0.49115	validation_1-auc:0.50429
[2]	validation_0-auc:0.50627	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



...

[0]	validation_0-auc:0.50407	validation_1-auc:0.50033
[1]	validation_0-auc:0.50599	validation_1-auc:0.56854
[2]	validation_0-auc:0.49968	validation_1-auc:0.44657
[3]	validation_0-auc:0.49104	validation_1-auc:0.50419


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.50035	validation_1-auc:0.50626
[1]	validation_0-auc:0.46181	validation_1-auc:0.50000
[2]	validation_0-auc:0.50027	validation_1-auc:0.49901


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.48398	validation_1-auc:0.50626
[1]	validation_0-auc:0.48397	validation_1-auc:0.50626
[2]	validation_0-auc:0.50950	validation_1-auc:0.50626
0.9238619727557499
0.0821214827268961


In [61]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["features"] = f_
df_xgb_TOPfeatures.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_xgb_TOPfeatures.csv")

In [62]:
len(TOP_features_XGB)

NameError: name 'TOP_features_XGB' is not defined

In [63]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

for i in range(len(features_list)-1):
    print(i)
    #features_list = list(TOPfeatures_XGB)[0:i+1]
    #features_list = list(TOP_features_XGB)#[0:i+1]
    features_list.remove(list(TOPfeatures_XGB)[-(i+1)])
    
    #df_sensitivity = df_maker(features_list=features_list, df= df_ROI_for_ML_Opti_ingested)
    #X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity)
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    #train_acr, test_acr = XGBoost_sensitivity(XTrain=X_train,XTest=X_test,yTrain=y_train,yTest=y_test)
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    #acr_his_XGB.append(train_acr)
    cv_auc_his_XGB.append(cv_auc_XGB)
    #acr_his_XGB.append(test_acr)
    cv_std_his_XGB.append(cv_std_XGB)
    #print(acr_his_XGB[-2:])
    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    try:
        #print(((acr_his_XGB[-2]-acr_his_XGB[-4])/acr_his_XGB[-4])*100)
        cv_change_XGB.append(((cv_auc_his_XGB[-1]-cv_auc_his_XGB[0])/cv_auc_his_XGB[0])*100)
        #print(((acr_his_XGB[-1]-acr_his_XGB[-3])/acr_his_XGB[-3])*100)
        auc_change_XGB.append(((c_auc_[-1]-c_auc_[0])/c_auc_[0])*100)
        f1_change_XGB.append(((c_f1_[-1]-c_f1_[0])/c_f1_[0])*100)
    except:
        pass

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.51660	validation_1-auc:0.50626
[1]	validation_0-auc:0.64241	validation_1-auc:0.47083
[2]	validation_0-auc:0.51143	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.50709	validation_1-auc:0.50626
[1]	validation_0-auc:0.49115	validation_1-auc:0.50429
[2]	validation_0-auc:0.50627	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



...

0.21505522758604506
0.07012322946577862
82
[0]	validation_0-auc:0.50062	validation_1-auc:0.50000
[1]	validation_0-auc:0.49787	validation_1-auc:0.50000
[2]	validation_0-auc:0.50085	validation_1-auc:0.50000
[0]	validation_0-auc:0.50366	validation_1-auc:0.50000
[1]	validation_0-auc:0.49795	validation_1-auc:0.50000
[2]	validation_0-auc:0.50364	validation_1-auc:0.50000
[0]	validation_0-auc:0.50216	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[1]	validation_0-auc:0.49762	validation_1-auc:0.50000
[2]	validation_0-auc:0.50085	validation_1-auc:0.50000
[0]	validation_0-auc:0.50061	validation_1-auc:0.50000
[1]	validation_0-auc:0.49758	validation_1-auc:0.50000
[2]	validation_0-auc:0.50143	validation_1-auc:0.50000
[0]	validation_0-auc:0.50275	validation_1-auc:0.50000
[1]	validation_0-auc:0.49813	validation_1-auc:0.50000
[2]	validation_0-auc:0.50083	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[0]	validation_0-auc:0.50083	validation_1-auc:0.50000
[1]	validation_0-auc:0.49909	validation_1-auc:0.50000
[2]	validation_0-auc:0.50091	validation_1-auc:0.50000
0.8041586078650568
0.1457007920567386


In [64]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["cv_change_XGB"] = cv_change_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["c_change_XGB_auc"] = auc_change_XGB
df_xgb_TOPfeatures["c_change_XGB_f1"] = f1_change_XGB
df_xgb_TOPfeatures.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/df_xgb_TOPfeatures_shortlisted.csv")

In [65]:
print(len(cv_auc_his_XGB), len(cv_std_his_XGB), len(cv_change_XGB), len(auc_change_XGB))

84 84 84 84


In [66]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

In [69]:
M_new_vec2 = np.array(cv_auc_his_XGB[::-1])
Sigma_new_vec2 = np.array(cv_std_his_XGB[::-1])

lower_bound2 = M_new_vec2 - Sigma_new_vec2
upper_bound2 = M_new_vec2 + Sigma_new_vec2
x_axis2 = np.arange(84)

plt.plot(x_axis2, M_new_vec2, 'go--', linewidth=0.5, markersize=0.1)
plt.fill_between(x_axis2, lower_bound2, upper_bound2, alpha=.3)
plt.plot(16, cv_auc_his_XGB[::-1][16], marker="o", markersize=5, markeredgecolor="blue", markerfacecolor="red")
plt.text(16, cv_auc_his_XGB[::-1][16], "the 17th feature", fontdict=dict(color="black", alpha=0.8, size=18))
plt.xlabel("Number of Features", size=16)
plt.ylabel("Average AUROC Score", size=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim((0,1))
plt.xlim((1,84))

plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_CV_Sensitivity.tif", bbox_inches = 'tight', dpi = 300)
plt.close()

In [70]:
#beCareful
TOP_features_XGB = list(TOPfeatures_XGB)[:17]

In [71]:
len(TOP_features_XGB)

17

In [72]:
TOP_features_XGB

['311.1684',
 '269.2486',
 '215.0328',
 '241.2173',
 '295.2278',
 '435.2965',
 '480.3096',
 '883.533',
 '309.1725',
 '339.1996',
 '233.1546',
 '337.2055',
 '265.1478',
 '279.2328',
 '790.5408',
 '303.2328',
 '325.1842']

In [73]:
features_list_XGB = list(TOP_features_XGB)

In [74]:
df_sensitivity_XGB = df_maker(features_list=features_list_XGB, df= df_ROI_for_ML_Opti_ingested)
X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity_XGB)

In [75]:
X_ingested = df_ROI_for_ML_Opti_ingested[features_list_XGB]

In [76]:
X_FNA = df_ROI_for_ML_Opti_FNA[features_list_XGB]

In [77]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list_XGB]

In [78]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    

    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set Accuracy:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_train2.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set Accuracy:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_val2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Test Set Accuracy:",metrics.roc_auc_score(yTest, predictions_FNA))
    print(classification_report(yTest,predictions_FNA))
    print(confusion_matrix(yTest,predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_ROC_test2.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/XGB_TopFeatures2.tif", bbox_inches = 'tight')
    #plt.show()


In [79]:
XGBoost_func()

[0]	validation_0-auc:0.50128	validation_1-auc:0.48549


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



[1]	validation_0-auc:0.49581	validation_1-auc:0.56854
[2]	validation_0-auc:0.51867	validation_1-auc:0.48978
Training Set Accuracy: 0.9300045382766526
              precision    recall  f1-score   support

           0       0.92      0.95      0.93     50392
           1       0.95      0.91      0.93     46643

    accuracy                           0.93     97035
   macro avg       0.93      0.93      0.93     97035
weighted avg       0.93      0.93      0.93     97035

[[47994  2398]
 [ 4310 42333]]
Validation Set Accuracy: 0.49580778901164674
              precision    recall  f1-score   support

           0       0.50      0.99      0.66     44540
           1       0.13      0.00      0.00     44161

    accuracy                           0.50     88701
   macro avg       0.31      0.50      0.33     88701
weighted avg       0.32      0.50      0.34     88701

[[44101   439]
 [44096    65]]
FNA Test Set Accuracy: 0.568537798921994
              precision    recall  f1-score   su

In [80]:
noSTDtop16 = pd.DataFrame()
noSTDtop16["top"] = list(features_list_XGB)
noSTDtop16.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/noSTDtop17.csv")

In [81]:
TOPfeatures_XGB_ = list(pd.read_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/5ppm/noSTDtop17.csv")["top"])
TOPfeatures_XGB = []
for ion in TOPfeatures_XGB_:
    TOPfeatures_XGB.append(str(ion))